In [5]:
# ライブラリ読み込み
import numpy as np
import matplotlib.pyplot as plt
import librosa
import glob
import json
from pydub import AudioSegment
from IPython.display import Audio
%matplotlib inline

from typing import List
from modules.network_interface import *

sampling_rate = 30000

In [2]:
# 音源読み込み
def load_audio(file_name: str) -> np.ndarray:
    return librosa.load(file_name, sr=sampling_rate, mono=True)[0]


# 読みデータ一括読み込み
def load_audios(lang: str) -> List[np.ndarray]:
    if lang != 'J' and lang != 'E':
        print("ファイルが見つかりません")
        return []
    
    audio_dir = './JKspeech/'
    file_names = glob.glob(f'{audio_dir}/{lang}*.wav')
    return [load_audio(file_name) for file_name in file_names]

# AudioSegmentに変換
def to_AudioSegment(file_name):
    try:
        audio_segment = AudioSegment.from_file(file_name, format="wav")
        return audio_segment
    except FileNotFoundError as e:
        print(e)
    return None


# 畳み込みで適合率を求める
def get_precision(origin_init: np.ndarray, target_init: np.ndarray) -> np.ndarray:
    # サイズ調整
    start = np.abs(len(origin_init) - len(target_init)) / 180
    start = np.min([int(start), 640])
    end = np.min([len(origin_init), len(target_init)])
    target = target_init[start:end]
    origin = origin_init[start:end]

    # フーリエ変換
    fft_O = np.fft.fft(origin)
    fft_T = np.fft.fft(target)

    # 畳み込み処理
    fft_Oc = np.conj(fft_O)        # 複素共役を取る
    sigma_C = fft_Oc * fft_T       # 要素ごとに乗算
    sigma_T = np.fft.ifft(sigma_C) # 逆フーリエ変換
    sigma_T = np.abs(sigma_T)      # 絶対値を取りスカラー値に変換
    
    return np.max(sigma_T)         # スカラー値の最大値 

In [6]:
# セットアップ
ja_sounds = load_audios('J')
en_sounds = load_audios('E')

---
##### ここまで済ませておく
---

In [ ]:
# 試合情報・問題情報の取得
match = await get_match()
print(match)

problem = await get_problem()
print(problem)

In [ ]:
# 制限時間表示
sec = problem["time_limit"]
print(f"制限時間: {int(sec/60)}分{sec%60:02d}秒")

# 取得する分割データの指定
chunks = await get_chunk(1)
print(chunks)

In [ ]:
# wavの取得
N = 2 # 取得する分割データ数

file_path = "./problems/"
chunks = (await get_chunk(N))["chunks"]
for chunk in chunks:
    await get_file(chunk, file_path)

In [ ]:
# 分割データ連結
file_path="./problems/"
N=5
chunks=[f"problem{i+1}.wav" for i in range(5)]
seg = to_AudioSegment(file_path + chunks.pop())
if N > 1:
    for chunk in chunks:
        seg += to_AudioSegment(file_path + chunk)
    
seg.export("./problems/tmp.wav", format="wav")
origin = load_audio("./problems/tmp.wav")

In [ ]:
# 畳み込み
precision = [get_precision(origin, sound) 
             for sound in ja_sounds + en_sounds]

In [ ]:
# ソート
answers = []
for i in np.argsort(precision)[::-1][:problem["data"]]:
    print((i%44)+1, precision[i])
    answers.append((i%44)+1)

answers = sorted(list(set(answers)))
print(answers)

In [ ]:
# 提出
formatted_answer = list(map(lambda n: f"{n:02d}", answers))
res = await answer(problem["id"], formatted_answer)

print(formatted_answer)
print(res)